# 05 - Vector Stores y Retrieval

## Curso de LLMs y Aplicaciones de IA

**Duración estimada:** 2-2.5 horas

---

## Índice

1. [Introducción a Vector Stores](#intro)
2. [FAISS: Vector Store local](#faiss)
3. [Document Loaders](#loaders)
4. [Text Splitters](#splitters)
5. [Similarity Search](#search)
6. [Persistencia y carga](#persistencia)
7. [Ejercicios prácticos](#ejercicios)

---

## Objetivos de aprendizaje

Al finalizar este notebook, serás capaz de:
- Entender qué son los vector stores y cómo funcionan
- Crear y usar FAISS para búsqueda vectorial
- Cargar documentos desde diferentes fuentes
- Dividir documentos en chunks óptimos
- Realizar búsquedas semánticas eficientes

<a name="intro"></a>
## 1. Introducción a Vector Stores

### ¿Qué es un Vector Store?

Un **Vector Store** (base de datos vectorial) es un sistema diseñado para almacenar, indexar y buscar vectores de alta dimensionalidad de forma eficiente.

### ¿Por qué son importantes?

Los LLMs tienen conocimiento limitado a su fecha de entrenamiento y no conocen datos privados/corporativos. Los vector stores permiten:

1. **Búsqueda semántica**: Encontrar documentos por significado, no keywords
2. **Memoria a largo plazo**: Almacenar conocimiento externo
3. **RAG**: Retrieval-Augmented Generation

### Opciones populares

| Vector Store | Tipo | Características |
|--------------|------|----------------|
| **FAISS** | Local | Gratuito, rápido, en memoria |
| **Chroma** | Local | Gratuito, persistente, fácil de usar |
| **Pinecone** | Cloud | Escalable, managed, pago |
| **Weaviate** | Cloud/Local | Open source, GraphQL |
| **Qdrant** | Cloud/Local | Open source, Rust |

Para este curso usamos **FAISS** por ser gratuito y no requerir infraestructura.

In [1]:
# Install required libraries (all free)
#%pip install -q langchain langchain-community langchain-huggingface
#%pip install -q faiss-cpu sentence-transformers
#%pip install -q beautifulsoup4 pypdf

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.1.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import warnings
warnings.filterwarnings('ignore')

print("Librerías instaladas ✓")

Librerías instaladas ✓


<a name="faiss"></a>
## 2. FAISS: Vector Store local

**FAISS** (Facebook AI Similarity Search) es una librería desarrollada por Meta para búsqueda eficiente de vectores similares.

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
#from langchain.docstore.document import Document
from langchain_core.documents import Document
# Load free embedding model
print("Cargando modelo de embeddings (gratuito)...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Test embedding
test_vector = embeddings.embed_query("Hello world")
print(f"Dimensión del embedding: {len(test_vector)}")

Cargando modelo de embeddings (gratuito)...


Loading weights: 100%|█████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 982.47it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Dimensión del embedding: 384


In [5]:
# Create simple documents
documents = [
    Document(
        page_content="Python es un lenguaje de programación interpretado y de alto nivel.",
        metadata={"source": "python.txt", "topic": "programming"}
    ),
    Document(
        page_content="JavaScript es el lenguaje de la web, ejecutado en navegadores.",
        metadata={"source": "javascript.txt", "topic": "programming"}
    ),
    Document(
        page_content="Machine Learning es un subcampo de la inteligencia artificial.",
        metadata={"source": "ml.txt", "topic": "AI"}
    ),
    Document(
        page_content="Deep Learning usa redes neuronales con múltiples capas.",
        metadata={"source": "dl.txt", "topic": "AI"}
    ),
    Document(
        page_content="FAISS es una librería para búsqueda eficiente de vectores similares.",
        metadata={"source": "faiss.txt", "topic": "tools"}
    ),
]

print(f"Creados {len(documents)} documentos")

Creados 5 documentos


In [6]:
# Create FAISS vector store
print("Creando vector store con FAISS...")
vector_store = FAISS.from_documents(documents, embeddings)

print(f"Vector store creado con {vector_store.index.ntotal} vectores")

Creando vector store con FAISS...
Vector store creado con 5 vectores


<a name="loaders"></a>
## 3. Document Loaders

LangChain proporciona loaders para cargar documentos desde múltiples fuentes.

In [7]:
# Web page loader
from langchain_community.document_loaders import WebBaseLoader

# Load content from a web page
print("Cargando contenido web...")
loader = WebBaseLoader("https://es.wikipedia.org/wiki/Python")
web_docs = loader.load()

print(f"Documentos cargados: {len(web_docs)}")
print(f"Caracteres: {len(web_docs[0].page_content)}")
print(f"Metadata: {web_docs[0].metadata}")
print(f"\nPrimeros 500 caracteres:\n{web_docs[0].page_content[:500]}...")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Cargando contenido web...
Documentos cargados: 1
Caracteres: 55474
Metadata: {'source': 'https://es.wikipedia.org/wiki/Python', 'title': 'Python - Wikipedia, la enciclopedia libre', 'language': 'es'}

Primeros 500 caracteres:




Python - Wikipedia, la enciclopedia libre
































Ir al contenido







Menú principal





Menú principal
mover a la barra lateral
ocultar



		Navegación
	


PortadaPortal de la comunidadActualidadCambios recientesPáginas nuevasPágina aleatoriaAyudaNotificar un errorPáginas especiales



















Buscar











Buscar






















Apariencia
















Donaciones

Crear una cuenta

Acceder








Herramientas personales





Donaciones Crea...


In [8]:
# Text file loader (create a sample file first)
sample_text = """Introducción a la Inteligencia Artificial

La inteligencia artificial (IA) es una rama de la informática que busca crear 
sistemas capaces de realizar tareas que normalmente requieren inteligencia humana.

Tipos de IA:
1. IA Débil (Narrow AI): Diseñada para tareas específicas
2. IA Fuerte (General AI): Capaz de cualquier tarea intelectual humana
3. Superinteligencia: Hipotética IA que supera la inteligencia humana

Aplicaciones comunes:
- Reconocimiento de voz
- Visión por computadora
- Procesamiento de lenguaje natural
- Sistemas de recomendación
"""

# Save to file
with open("sample_ai_doc.txt", "w", encoding="utf-8") as f:
    f.write(sample_text)

# Load from file
from langchain_community.document_loaders import TextLoader

text_loader = TextLoader("sample_ai_doc.txt", encoding="utf-8")
text_docs = text_loader.load()

print(f"Documento cargado: {len(text_docs)} archivo(s)")
print(f"Contenido:\n{text_docs[0].page_content}")

Documento cargado: 1 archivo(s)
Contenido:
Introducción a la Inteligencia Artificial

La inteligencia artificial (IA) es una rama de la informática que busca crear 
sistemas capaces de realizar tareas que normalmente requieren inteligencia humana.

Tipos de IA:
1. IA Débil (Narrow AI): Diseñada para tareas específicas
2. IA Fuerte (General AI): Capaz de cualquier tarea intelectual humana
3. Superinteligencia: Hipotética IA que supera la inteligencia humana

Aplicaciones comunes:
- Reconocimiento de voz
- Visión por computadora
- Procesamiento de lenguaje natural
- Sistemas de recomendación



<a name="splitters"></a>
## 4. Text Splitters

Los documentos largos deben dividirse en **chunks** (fragmentos) para:
- Respetar límites de contexto del LLM
- Mejorar precisión de búsqueda
- Reducir costos de procesamiento

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,        # Maximum characters per chunk
    chunk_overlap=50,      # Overlap between chunks
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Priority of separators
)

# Split the document
chunks = text_splitter.split_documents(text_docs)

print(f"Documento original dividido en {len(chunks)} chunks\n")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} ({len(chunk.page_content)} chars):")
    print(f"  '{chunk.page_content[:80]}...'")
    print()

Documento original dividido en 5 chunks

Chunk 1 (41 chars):
  'Introducción a la Inteligencia Artificial...'

Chunk 2 (161 chars):
  'La inteligencia artificial (IA) es una rama de la informática que busca crear 
s...'

Chunk 3 (141 chars):
  'Tipos de IA:
1. IA Débil (Narrow AI): Diseñada para tareas específicas
2. IA Fue...'

Chunk 4 (69 chars):
  '3. Superinteligencia: Hipotética IA que supera la inteligencia humana...'

Chunk 5 (134 chars):
  'Aplicaciones comunes:
- Reconocimiento de voz
- Visión por computadora
- Procesa...'



In [12]:
# Different chunk sizes comparison
sizes = [100, 200, 500]

long_text = web_docs[0].page_content[:2000]  # First 2000 chars from Wikipedia

print("Comparación de tamaños de chunk:")
print("=" * 50)

for size in sizes:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=size // 5  # 20% overlap
    )
    chunks = splitter.split_text(long_text)
    print(f"chunk_size={size}: {len(chunks)} chunks")

Comparación de tamaños de chunk:
chunk_size=100: 26 chunks
chunk_size=200: 13 chunks
chunk_size=500: 5 chunks


### Estrategia de chunking

| Chunk Size | Ventajas | Desventajas |
|------------|----------|-------------|
| Pequeño (100-300) | Más preciso, específico | Puede perder contexto |
| Mediano (300-800) | Balance contexto/precisión | Uso general |
| Grande (800-2000) | Más contexto | Menos preciso, más tokens |

<a name="search"></a>
## 5. Similarity Search

La búsqueda por similitud es la operación fundamental de los vector stores.

In [13]:
# Create vector store with more documents
all_chunks = text_splitter.split_documents(text_docs)

# Add more sample documents
extra_docs = [
    Document(page_content="Los transformers revolucionaron el NLP en 2017.", 
             metadata={"topic": "NLP"}),
    Document(page_content="GPT-4 es un modelo de lenguaje desarrollado por OpenAI.",
             metadata={"topic": "LLM"}),
    Document(page_content="BERT es un modelo bidireccional pre-entrenado.",
             metadata={"topic": "NLP"}),
    Document(page_content="Las redes convolucionales son excelentes para imágenes.",
             metadata={"topic": "CV"}),
    Document(page_content="El aprendizaje por refuerzo entrena agentes mediante recompensas.",
             metadata={"topic": "RL"}),
]

all_docs = all_chunks + extra_docs

# Create new vector store
vs = FAISS.from_documents(all_docs, embeddings)
print(f"Vector store con {vs.index.ntotal} documentos")

Vector store con 10 documentos


In [14]:
# Basic similarity search
query = "¿Qué es el procesamiento de lenguaje natural?"

results = vs.similarity_search(query, k=3)

print(f"Query: '{query}'")
print("\nResultados:")
print("=" * 50)
for i, doc in enumerate(results, 1):
    print(f"\n{i}. {doc.page_content}")
    print(f"   Metadata: {doc.metadata}")

Query: '¿Qué es el procesamiento de lenguaje natural?'

Resultados:

1. BERT es un modelo bidireccional pre-entrenado.
   Metadata: {'topic': 'NLP'}

2. GPT-4 es un modelo de lenguaje desarrollado por OpenAI.
   Metadata: {'topic': 'LLM'}

3. Las redes convolucionales son excelentes para imágenes.
   Metadata: {'topic': 'CV'}


In [15]:
# Similarity search with scores
results_with_scores = vs.similarity_search_with_score(query, k=3)

print(f"Query: '{query}'")
print("\nResultados con puntuación (menor = más similar):")
print("=" * 50)
for doc, score in results_with_scores:
    print(f"\nScore: {score:.4f}")
    print(f"  {doc.page_content}")

Query: '¿Qué es el procesamiento de lenguaje natural?'

Resultados con puntuación (menor = más similar):

Score: 1.1154
  BERT es un modelo bidireccional pre-entrenado.

Score: 1.1231
  GPT-4 es un modelo de lenguaje desarrollado por OpenAI.

Score: 1.1309
  Las redes convolucionales son excelentes para imágenes.


In [16]:
# Search with metadata filter
# Note: FAISS doesn't support native filtering, but we can post-filter

def search_with_filter(vector_store, query, k=10, filter_key=None, filter_value=None):
    """Search with optional metadata filtering."""
    # Get more results to filter
    results = vector_store.similarity_search(query, k=k)
    
    if filter_key and filter_value:
        results = [doc for doc in results 
                   if doc.metadata.get(filter_key) == filter_value]
    
    return results

# Search only NLP documents
nlp_results = search_with_filter(vs, "modelos de lenguaje", filter_key="topic", filter_value="NLP")

print("Búsqueda filtrada (topic='NLP'):")
for doc in nlp_results[:3]:
    print(f"  - {doc.page_content}")

Búsqueda filtrada (topic='NLP'):
  - BERT es un modelo bidireccional pre-entrenado.
  - Los transformers revolucionaron el NLP en 2017.


### Retriever: Interfaz para búsqueda

LangChain proporciona una interfaz `Retriever` para usar vector stores en chains.

In [17]:
# Create retriever from vector store
retriever = vs.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# Use retriever
docs = retriever.invoke("inteligencia artificial")

print(f"Retriever devolvió {len(docs)} documentos:")
for doc in docs:
    print(f"  - {doc.page_content[:60]}...")

Retriever devolvió 3 documentos:
  - La inteligencia artificial (IA) es una rama de la informátic...
  - Introducción a la Inteligencia Artificial...
  - 3. Superinteligencia: Hipotética IA que supera la inteligenc...


<a name="persistencia"></a>
## 6. Persistencia y carga

FAISS permite guardar y cargar el índice para no tener que recalcular embeddings.

In [18]:
# Save vector store to disk
vs.save_local("faiss_index")
print("Vector store guardado en 'faiss_index/'")

Vector store guardado en 'faiss_index/'


In [19]:
# Load vector store from disk
loaded_vs = FAISS.load_local(
    "faiss_index", 
    embeddings,
    allow_dangerous_deserialization=True  # Required for pickle files
)

print(f"Vector store cargado con {loaded_vs.index.ntotal} vectores")

# Test that it works
test_results = loaded_vs.similarity_search("IA", k=1)
print(f"Test búsqueda: {test_results[0].page_content[:50]}...")

Vector store cargado con 10 vectores
Test búsqueda: Tipos de IA:
1. IA Débil (Narrow AI): Diseñada par...


In [20]:
# Add more documents to existing vector store
new_docs = [
    Document(page_content="LangChain facilita el desarrollo de aplicaciones con LLMs.",
             metadata={"topic": "tools"}),
    Document(page_content="Hugging Face es la plataforma líder para modelos de ML.",
             metadata={"topic": "tools"}),
]

# Add to vector store
loaded_vs.add_documents(new_docs)

print(f"Vectores después de añadir: {loaded_vs.index.ntotal}")

Vectores después de añadir: 12


<a name="ejercicios"></a>
## 7. Ejercicios Prácticos

### Ejercicio 1: Crear un buscador de FAQs

In [21]:
# Exercise 1: FAQ Search System

faqs = [
    {"q": "¿Cómo puedo resetear mi contraseña?", 
     "a": "Ve a 'Olvidé mi contraseña' en la página de login."},
    {"q": "¿Cuál es el horario de atención?",
     "a": "Lunes a Viernes de 9:00 a 18:00."},
    {"q": "¿Aceptan devoluciones?",
     "a": "Sí, tienes 30 días para devolver productos sin usar."},
    {"q": "¿Tienen envío internacional?",
     "a": "Sí, enviamos a más de 50 países."},
    {"q": "¿Cómo contacto con soporte?",
     "a": "Email: soporte@ejemplo.com o chat en vivo."},
]

# Create documents with questions and answers
faq_docs = [
    Document(
        page_content=faq["q"],
        metadata={"answer": faq["a"]}
    )
    for faq in faqs
]

# Create vector store
faq_vs = FAISS.from_documents(faq_docs, embeddings)

# Search function
def find_answer(query):
    results = faq_vs.similarity_search(query, k=1)
    if results:
        return results[0].metadata["answer"]
    return "No encontré una respuesta."

# Test
test_queries = [
    "olvidé mi clave",
    "quiero devolver algo",
    "¿envían a México?"
]

print("Sistema FAQ:")
for q in test_queries:
    print(f"Q: {q}")
    print(f"A: {find_answer(q)}\n")

Sistema FAQ:
Q: olvidé mi clave
A: Lunes a Viernes de 9:00 a 18:00.

Q: quiero devolver algo
A: Lunes a Viernes de 9:00 a 18:00.

Q: ¿envían a México?
A: Sí, enviamos a más de 50 países.



### Ejercicio 2: Comparar diferentes chunk sizes

In [22]:
# Exercise 2: Experiment with chunk sizes
# Load a longer document and test different chunk sizes

# Create a longer sample document
long_document = """
La inteligencia artificial ha transformado numerosas industrias en las últimas décadas.
Desde el reconocimiento de voz hasta los vehículos autónomos, las aplicaciones son vastas.

En el sector salud, la IA ayuda a diagnosticar enfermedades con mayor precisión.
Los algoritmos pueden analizar imágenes médicas y detectar anomalías que los humanos podrían pasar por alto.

En finanzas, los modelos predictivos ayudan a detectar fraudes y evaluar riesgos crediticios.
Los chatbots atienden consultas de clientes las 24 horas del día.

El comercio electrónico utiliza IA para personalizar recomendaciones de productos.
Los sistemas analizan el historial de compras y navegación para sugerir artículos relevantes.

Sin embargo, la IA también plantea desafíos éticos importantes.
La privacidad de datos, el sesgo algorítmico y el desplazamiento laboral son temas críticos.
"""

# Test different chunk sizes and see search quality
chunk_sizes = [100, 200, 400]
query = "aplicaciones de IA en salud"

print(f"Query: '{query}'\n")

for size in chunk_sizes:
    splitter = RecursiveCharacterTextSplitter(chunk_size=size, chunk_overlap=20)
    chunks = splitter.create_documents([long_document])
    vs_test = FAISS.from_documents(chunks, embeddings)
    results = vs_test.similarity_search(query, k=1)
    
    print(f"Chunk size={size}: {len(chunks)} chunks")
    print(f"  Mejor resultado: '{results[0].page_content[:60]}...'\n")

Query: 'aplicaciones de IA en salud'

Chunk size=100: 11 chunks
  Mejor resultado: 'En el sector salud, la IA ayuda a diagnosticar enfermedades ...'

Chunk size=200: 5 chunks
  Mejor resultado: 'El comercio electrónico utiliza IA para personalizar recomen...'

Chunk size=400: 3 chunks
  Mejor resultado: 'En finanzas, los modelos predictivos ayudan a detectar fraud...'



## Resumen

En este notebook hemos aprendido:

1. **Vector Stores**: Bases de datos para búsqueda semántica
2. **FAISS**: Vector store local, rápido y gratuito
3. **Document Loaders**: Cargar desde archivos, web, PDFs
4. **Text Splitters**: Dividir documentos en chunks
5. **Similarity Search**: Buscar por significado
6. **Persistencia**: Guardar y cargar índices

### Arquitectura típica de un sistema de retrieval

```
Documentos → [Loader] → [Splitter] → [Embeddings] → [Vector Store]
                                                         ↓
Query → [Embedding] → [Similarity Search] → Documentos relevantes
```

En el siguiente notebook veremos cómo combinar esto con LLMs para crear sistemas **RAG** (Retrieval-Augmented Generation).

---

## Referencias

- [FAISS Documentation](https://faiss.ai/)
- [LangChain Vector Stores](https://python.langchain.com/docs/modules/data_connection/vectorstores/)
- [Chunking Strategies](https://www.pinecone.io/learn/chunking-strategies/)

In [23]:
import session_info
session_info.show(html = False)

-----
langchain_community         0.4.1
langchain_core              1.2.9
langchain_huggingface       NA
langchain_text_splitters    NA
session_info                v1.0.1
-----
IPython             9.10.0
jupyter_client      8.8.0
jupyter_core        5.9.1
-----
Python 3.13.1 (tags/v3.13.1:0671451, Dec  3 2024, 19:06:28) [MSC v.1942 64 bit (AMD64)]
Windows-11-10.0.26200-SP0
-----
Session information updated at 2026-02-09 16:07
